### IMDB Sentiment Analysis
Binary classification project using a Recurrent Neural Network with LSTM architecture to predict movie review sentiment.

Dataset can be found on Kaggle (https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews) and contains 50K movie reviews labelled as positive or negative.

This [Medium article]("https://medium.com/@muhammadluay45/sentiment-analysis-using-recurrent-neural-network-rnn-long-short-term-memory-lstm-and-38d6e670173f") was used as reference for project direction.

In [26]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from keras.regularizers import l2
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np

### Preprocessing data

In [2]:
dataset = pd.read_csv("IMDB_Dataset.csv")

In [3]:
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
dataset['review'][0]
#to look at the general content and length of a review

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [9]:
dataset["sentiment"].value_counts()
#this dataset is balanced as it has an equalsplit for positive and negative sentiments

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [28]:
dataset.isnull().sum()
#no missing values

review       0
sentiment    0
dtype: int64

#### Cleaning the data

In [10]:
import re
def clean_text(text):
    text = text.lower()
    # Remove all characters except lowercase alphabets and spaces
    text = re.sub(r'[^a-z\s]', '', text)
    return text

dataset["review"] = dataset["review"].apply(clean_text)

In [11]:
dataset["review"][0]

'one of the other reviewers has mentioned that after watching just  oz episode youll be hooked they are right as this is exactly what happened with mebr br the first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the wordbr br it is called oz as that is the nickname given to the oswald maximum security state penitentary it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda em city is home to manyaryans muslims gangstas latinos christians italians irish and moreso scuffles death stares dodgy dealings and shady agreements are never far awaybr br i would say the main appeal of the show is due to the fact that it goes where other shows wouldnt dare for

In [12]:
dataset.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production br br the filmin...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically theres a family where a little boy j...,negative
4,petter matteis love in the time of money is a ...,positive


### One hot encoding

In [13]:
#one hot encoding for labels
data_encoded = pd.get_dummies(dataset, columns=["sentiment", ], drop_first=True, dtype=int)

In [14]:
data_encoded.head()
#positive values are reresented by 1, and negative values are represented by 0

,review,sentiment_positive
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production br br the filmin...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically theres a family where a little boy j...,0
4,petter matteis love in the time of money is a ...,1


In [15]:
data_encoded = data_encoded.rename(columns={"sentiment_positive": "sentiment"})

In [16]:
data_encoded["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

### Splitting into training and testing data

In [17]:
train, test = train_test_split(data_encoded, test_size=0.2, random_state=24)
#only 2 columns in the data so no need to divide the dataset further into X and Y here

In [18]:
train.shape

(40000, 2)

In [19]:
test.shape

(10000, 2)

In [20]:
tokenizer = Tokenizer(num_words=5000) #IDs of the 5000 most frequent words are returned
tokenizer.fit_on_texts(train["review"])

In [21]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train["review"]), maxlen=300)
X_test = pad_sequences(tokenizer.texts_to_sequences(test["review"]), maxlen=300)

In [22]:
X_train
#array of tekenized text values

array([[   0,    0,    0, ...,  210,   26,  105],
       [   0,    0,    0, ...,    1,   12,  372],
       [  30,    1,  162, ...,    7,    3, 3284],
       ...,
       [   0,    0,    0, ...,    3,  225,  334],
       [   0,    0,    0, ...,  224,  259,    5],
       [   0,    0,    0, ..., 1465,  105,    8]])

In [23]:
Y_train = train["sentiment"]
Y_test = test["sentiment"]

In [24]:
Y_train

31724    1
9389     1
17228    1
41843    0
31584    0
        ..
21633    0
19857    1
14528    1
899      0
45474    1
Name: sentiment, Length: 40000, dtype: int32

### Build model 

Bidirectional LSTM layers were included instead of basic LSTM layers, as they can improve the model's ability to understand context in sequences by processing the input in both forward and backward directions.  To add complexity, L2 regularization was used to apply penalities on the Dense layer's kernel, and dropout layers were added to avoid overfitting. These additions will allow the model to learn more complex patterns. 

In [27]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=256)) # increased embedding dimension from 128
model.add(Bidirectional(LSTM(128, return_sequences=True, dropout=0.20, recurrent_dropout=0.20)))
model.add(Bidirectional(LSTM(128, dropout=0.20, recurrent_dropout=0.20))) 
model.add(Dropout(0.2)) 
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01))) # Dense layer with L2 regularization
model.add(Dropout(0.2)) 
model.add(Dense(1, activation='sigmoid'))

In [29]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [30]:
model.fit(X_train, Y_train, epochs=5, batch_size=16, validation_split=0.2)

Epoch 1/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 793s 394ms/step - accuracy: 0.5375 - loss: 0.7880 - val_accuracy: 0.8181 - val_loss: 0.4295
Epoch 2/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 818s 409ms/step - accuracy: 0.8335 - loss: 0.4129 - val_accuracy: 0.8849 - val_loss: 0.3119
Epoch 3/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 840s 420ms/step - accuracy: 0.8953 - loss: 0.2882 - val_accuracy: 0.8860 - val_loss: 0.2905
Epoch 4/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 823s 411ms/step - accuracy: 0.9094 - loss: 0.2499 - val_accuracy: 0.8910 - val_loss: 0.2881
Epoch 5/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 849s 425ms/step - accuracy: 0.9288 - loss: 0.2068 - val_accuracy: 0.8966 - val_loss: 0.2967


In [32]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (16, 300, 256)              │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (16, 300, 256)              │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (16, 256)                   │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (16, 256)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (16, 64)                    │          16,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (16, 64)                    │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (16, 1)                     │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,254,981 (23.86 MB)

 Trainable params: 2,084,993 (7.95 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,169,988 (15.91 MB)

In [33]:
model.save("sentiment_model.keras")

In [34]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

### Evaluate the model

In [35]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 69s 219ms/step - accuracy: 0.8911 - loss: 0.3163


In [36]:
print(f'Accuracy: {accuracy}, Loss: {loss}')

Accuracy: 0.8916000127792358, Loss: 0.31192877888679504


### Predictions

In [37]:
def sentiment_prediction(review):
    seq = tokenizer.texts_to_sequences([review]) 
    padded_seq = pad_sequences(seq, maxlen=300)
    
    prediction = model.predict(padded_seq)
    #prediction is a scole between 0 and 1, with 0 being a negative review and 1 being a positive review
    
    if prediction[0][0] > 0.6: 
        sentiment = "This is a positive review."
    elif prediction[0][0] < 0.4:
        sentiment = "This is a negative review."
    else:
        sentiment = "This review is not strongly negative or positive."
    
    return sentiment

In [50]:
sentiment_prediction("It was my favourite movie") 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


'This is a positive review.'

In [39]:
sentiment_prediction("I hated every minute of this movie")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


'This is a negative review.'

In [40]:
sentiment_prediction("The costuming in this movie was great, and the art direction was some of my favourite")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


'This is a positive review.'

In [41]:
sentiment_prediction("This movie is fantastic")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


'This is a positive review.'